Excellent use of `@model_validator` in **Pydantic v2**! Let’s break it down step by step so you deeply understand what `@model_validator` does, how it compares to `@field_validator`, and how your current example works under the hood.

---

## 🔍 What is `@model_validator`?

`@model_validator` is a Pydantic v2 decorator used to **validate or modify the entire model instance** — not just individual fields.

---

## 🔧 Syntax Overview

```python
from pydantic import model_validator

@model_validator(mode='before' or 'after')
def method_name(cls, model):
    # custom logic
    return model
```

---

## 📌 `mode` Options

| Mode     | Input Passed To Validator | When it Runs                       |
| -------- | ------------------------- | ---------------------------------- |
| `before` | `dict` of raw input data  | Before field validation/conversion |
| `after`  | A fully parsed model      | After all fields are validated     |

---

## ✅ Use Case You Gave

```python
class Patient(BaseModel):

    name: str
    email: EmailStr
    age: int
    weight: float
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

    @model_validator(mode='after')
    def validate_emergency_contact(cls, model):
        if model.age > 60 and 'emergency' not in model.contact_details:
            raise ValueError('Patients older than 60 must have an emergency contact')
        return model
```

---

### 🔎 What's Happening Internally?

1. **Input is parsed**:

   * `age` (a string: `'65'`) is converted to `int` → `65`.
   * `email` is validated as a proper `EmailStr`.
   * List and dict fields are parsed and type-checked.

2. **Then `@model_validator(mode='after')` is triggered**:

   * It receives the full instance: `model: Patient`
   * You apply **cross-field logic**:

     * If `age > 60`, then `"emergency"` **must** be in `contact_details`.

3. If this condition is violated:

   * Raises `ValueError` with a message.
   * Model initialization fails.

---

### 🧪 Input Example You Used

```python
patient_info = {
    'name': 'nitish',
    'email': 'abc@icici.com',
    'age': '65',
    'weight': 75.2,
    'married': True,
    'allergies': ['pollen', 'dust'],
    'contact_details': {
        'phone': '2353462',
        'emergency': '235236'
    }
}
```

* `age = 65` → valid
* `"emergency"` is present → ✅ passes validation

### Output:

```text
NITISH
65
['pollen', 'dust']
True
updated
```

---

## ❌ What If Emergency Contact is Missing?

```python
patient_info = {
    'name': 'nitish',
    'email': 'abc@icici.com',
    'age': 70,
    'weight': 75.2,
    'married': True,
    'allergies': ['pollen', 'dust'],
    'contact_details': {
        'phone': '2353462'
    }
}

Patient(**patient_info)
```

### Output:

```text
pydantic_core._pydantic_core.ValidationError: 1 validation error for Patient
  Patients older than 60 must have an emergency contact (type=value_error)
```

---

## 🔁 Model vs Field Validator

| Feature                | `@field_validator`             | `@model_validator`               |
| ---------------------- | ------------------------------ | -------------------------------- |
| Scope                  | One or more fields             | Entire model                     |
| Mode                   | `before`, `after`, `wrap`      | `before`, `after`                |
| Use Case               | Type checks, field-level rules | Cross-field logic, global checks |
| Example                | Email domain check             | Emergency contact if age > 60    |
| Input passed to method | Field value                    | Whole model or dict              |

---

## ✅ Best Practices for `@model_validator`

* Use `mode='after'` when you need **fully parsed** field values (like `int` instead of raw `'65'`).
* Use `mode='before'` if you want to:

  * Modify input keys/values,
  * Fill defaults based on raw data.

---

## 🔧 Bonus Example (Using `mode='before'`)

```python
@model_validator(mode='before')
@classmethod
def fill_emergency_if_missing(cls, data):
    if data.get('age', 0) > 60 and 'emergency' not in data.get('contact_details', {}):
        data['contact_details']['emergency'] = 'N/A'
    return data
```

This **auto-fills** `"emergency"` if missing — before parsing even begins.

---

## ✅ Summary

| Concept       | Description                                                      |
| ------------- | ---------------------------------------------------------------- |
| Purpose       | Validate or modify the entire model                              |
| Mode `after`  | Used after all fields are validated and type-coerced             |
| Mode `before` | Used on raw data (dict), allows mutation before field validation |
| Access        | You get access to either the raw input or the entire model       |
| Use Cases     | Cross-field dependencies, required-if-conditions, derived logic  |

---

Would you like me to combine both `@field_validator` and `@model_validator` into one real-world example like:

* Age-based rules,
* Email domain check,
* Auto-fill missing fields,
* Or FastAPI + schema generation using this model?

Let me know!
